In [35]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import os

# 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = BertForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
labels = ["negative", "neutral", "positive"]

def get_SA_result(target_text):
    SA_result_dict = {}

    # 입력 텍스트 토크나이즈 (max_length 설정)
    inputs = tokenizer(target_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # 모델로 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 출력 값 확인 (로짓)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1)

    # 클래스 레이블
    predicted_label = labels[predicted_class.item()]
    SA_result_dict['label'] = predicted_label

    # 예측 확률 출력
    probabilities = torch.softmax(logits, dim=-1)
    predicted_probability = probabilities[0][predicted_class].item()
    SA_result_dict['prob'] = predicted_probability

    return SA_result_dict

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

tickers = [file for file in os.listdir('../../../store_data/raw/crawling/corp_rel_news/') if file != '.DS_Store']

for i, ticker in enumerate(tickers):
    original_parent_path = f'../../../store_data/raw/crawling/corp_rel_news/{ticker}'
    years_list = sorted([file for file in os.listdir(original_parent_path) if file != '.DS_Store'])

    print(f'{i+1} / {len(tickers)}')
    for year in years_list:
        print(f'===== {year}에 대해 감성분석 시작! =====')
        for quarter in quarters:
            try:
                print(f'===== {ticker} {year}의 {quarter}에 대해 처리중... =====')
                if quarter == 'Q1':
                    month = '01'
                elif quarter == 'Q2':
                    month = '04'
                elif quarter == 'Q3':
                    month = '07'
                elif quarter == 'Q4':
                    month = '10'
                else:
                    print(f'입력된 quarter가 올바르지 않습니다 | 입력된 quarter: {quarter}')

                original_path_by_date = os.path.join(original_parent_path, year, month)
                file_list = os.listdir(original_path_by_date)
                if len(file_list) != 1:
                    print(f'크롤링 결과 확인 필요: {file_list}')
                
                csv_path = os.path.join(original_path_by_date, file_list[0])
                df = pd.read_csv(csv_path)

                temp_SA_label = []
                temp_SA_prob = []

                for i in range(len(df)):
                    target_text = df.iloc[i]['news_title']
                    SA_result = get_SA_result(target_text)

                    temp_SA_label.append(SA_result['label'])
                    temp_SA_prob.append(SA_result['prob'])

                df['label'] = temp_SA_label
                df['prob'] = temp_SA_prob
                
                save_path = original_path_by_date.replace('/raw/', '/process/')
                os.makedirs(save_path, exist_ok=True)
                df.to_csv(os.path.join(save_path, f'SA_{file_list[0]}'))
            except Exception as e:
                print(f'{year} {quarter} {ticker}에 대해 처리 중 오류 발생! | {e}')

1 / 901
===== 2019에 대해 감성분석 시작! =====
===== 047040 2019의 Q1에 대해 처리중... =====
===== 047040 2019의 Q2에 대해 처리중... =====
===== 047040 2019의 Q3에 대해 처리중... =====
===== 047040 2019의 Q4에 대해 처리중... =====
===== 2020에 대해 감성분석 시작! =====
===== 047040 2020의 Q1에 대해 처리중... =====
===== 047040 2020의 Q2에 대해 처리중... =====
2020 Q2 047040에 대해 처리 중 오류 발생! | [Errno 2] No such file or directory: '../../../store_data/raw/crawling/corp_rel_news/047040/2020/04'
===== 047040 2020의 Q3에 대해 처리중... =====
2020 Q3 047040에 대해 처리 중 오류 발생! | [Errno 2] No such file or directory: '../../../store_data/raw/crawling/corp_rel_news/047040/2020/07'
===== 047040 2020의 Q4에 대해 처리중... =====
2020 Q4 047040에 대해 처리 중 오류 발생! | [Errno 2] No such file or directory: '../../../store_data/raw/crawling/corp_rel_news/047040/2020/10'
===== 2021에 대해 감성분석 시작! =====
===== 047040 2021의 Q1에 대해 처리중... =====
===== 047040 2021의 Q2에 대해 처리중... =====
===== 047040 2021의 Q3에 대해 처리중... =====
===== 047040 2021의 Q4에 대해 처리중... =====
===== 2022에 대해 감성분석 시작! =====
==

NotADirectoryError: [Errno 20] Not a directory: '../../../store_data/raw/crawling/corp_rel_news/temptext'